In [8]:
from torch.quasirandom import SobolEngine
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf
import hydra
import torch
import botorch
import wandb
from botorch.models.transforms import Standardize
from botorch.models.transforms.input import Normalize
from mgp_models.fully_bayesian import  MGPFullyBayesianSingleTaskGP
from mgp_models.fit_fully_bayesian import fit_fully_bayesian_mgp_model_nuts, fit_partially_bayesian_mgp_model
from mgp_models.utils import get_candidate_pool, get_test_set, get_acq_values_pool, eval_nll, eval_rmse, convert_bounds
from mgp_models.acquisition import BQBCAcquisitionFunction
import time

In [15]:
import math
from typing import Optional

from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.acquisition import AnalyticAcquisitionFunction
from botorch.models.model import Model
from botorch.sampling.base import MCSampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils import t_batch_mode_transform
from torch import Tensor
import torch

from botorch.acquisition import AnalyticAcquisitionFunction
from mgp_models.fully_bayesian import  MGPFullyBayesianSingleTaskGP
from botorch.posteriors.fully_bayesian import GaussianMixturePosterior, MCMC_DIM


In [9]:
tkwargs = {
"device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
"dtype": torch.double,
}

In [10]:
DIM = 6
BOUNDS = [[0,1], [0,1], [0,1],[0,1],[0,1],[0,1]]

In [31]:
synthetic_function = botorch.test_functions.Hartmann(noise_std=0.13784048).to(**tkwargs)
bounds = synthetic_function.bounds
    #print(bounds)
X = SobolEngine(dimension=6, scramble=True, seed=0).draw(30).to(**tkwargs)
    #print(X)
X_scaled = convert_bounds(X, BOUNDS, DIM)
Y = synthetic_function(X_scaled).unsqueeze(-1)

In [32]:
train_Y = Y  # Flip the sign since we want to minimize f(x)
gp = MGPFullyBayesianSingleTaskGP(
    train_X=X, 
    train_Y=train_Y, 
    #train_Yvar=torch.full_like(train_Y, 1e-6),
    #input_transform=Normalize(d=cfg.functions.dim, bounds=bountensor_scaledds),
    outcome_transform=Standardize(m=1)
)
ll = fit_partially_bayesian_mgp_model(gp,
                                        20,
                                        0.1,
                                        300,
                                        print_iter=False)

In [52]:
acq_function = SALHRMMAcquisitionFunction(gp, ll=ll)

poolU = get_candidate_pool(dim=DIM, bounds=BOUNDS, size=10).to(**tkwargs)
acq_values = acq_function(poolU)
acq_values

tensor([[[0.9966],
         [0.9999],
         [0.9844],
         [0.9714],
         [0.9804],
         [0.9984],
         [0.9901],
         [0.9803],
         [0.9869],
         [0.9853]],

        [[0.9915],
         [0.9995],
         [0.9664],
         [0.9460],
         [0.9756],
         [0.9962],
         [0.9678],
         [0.9390],
         [0.9670],
         [0.9761]],

        [[0.9962],
         [0.9999],
         [0.9828],
         [0.9685],
         [0.9784],
         [0.9982],
         [0.9891],
         [0.9783],
         [0.9855],
         [0.9838]],

        [[0.9863],
         [0.9995],
         [0.9432],
         [0.9020],
         [0.9351],
         [0.9880],
         [0.9668],
         [0.9319],
         [0.9518],
         [0.9454]],

        [[0.9869],
         [0.9991],
         [0.9453],
         [0.9049],
         [0.9535],
         [0.9944],
         [0.9664],
         [0.9328],
         [0.9508],
         [0.9497]],

        [[0.9889],
         [0.9997],
  

In [53]:
class SALHRMMAcquisitionFunction(AnalyticAcquisitionFunction):
    def __init__(
        self,
        model: MGPFullyBayesianSingleTaskGP,
        maximize: bool = True,
        ll: Optional[Tensor] = None
    ) -> None:
        # we use the AcquisitionFunction constructor, since that of
        # AnalyticAcquisitionFunction performs some validity checks that we don't want here
        super(AnalyticAcquisitionFunction, self).__init__(model)
        self.maximize = maximize
        self.ll = ll

    def forward(self, X: Tensor) -> Tensor:


        posterior = self.model.posterior(X, ll= self.ll)
        n_models = posterior._mean.shape[MCMC_DIM]
        mean_minus_mgpmean = posterior._mean - posterior.mixture_mean.repeat(n_models,1,1)
        BQBC = mean_minus_mgpmean.pow(2).sum(dim=MCMC_DIM)
        var = posterior._variance.sum(dim=MCMC_DIM)
        mixture_variance = BQBC + var
        sigma_1 = mixture_variance.repeat(n_models,1,1)
        mixture_mean = posterior._mean.sum(dim=MCMC_DIM)
        mu_1 = mixture_mean.repeat(n_models,1,1)
        sigma_2 = posterior.variance
        mu_2 = posterior.mean
        up = 2*torch.sqrt(sigma_1)*torch.sqrt(sigma_2)
        down = sigma_1+sigma_2
        to_sqrt = up.div(down)
        sqrted = torch.sqrt(to_sqrt)
        mean_up = mu_1 - mu_2
        mean_up = mean_up.pow(2)
        exped = torch.exp(-0.25*mean_up.div(down))
        right = sqrted* exped
        hellinger = 1 - right
        return hellinger.mul(posterior.shaped_weights).sum(dim=MCMC_DIM)
        

